In [1]:
import pandas as pd
import numpy as np

In [2]:
df1=pd.read_csv('./train_essays_RDizzl3_seven_v1.csv')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
df1.head(2)

,text,label
0,Cars. Cars have been around since they became ...,0
1,Transportation is a large necessity in most co...,0


In [ ]:

df1['label'].value_counts()

0    14250
1     1621
Name: label, dtype: int64

In [ ]:
df2 = pd.read_csv("./daigt_external_dataset.csv", sep=',')
df2 = df2[["source_text"]]
df2.columns = ["text"]
df2['text'] = df2['text'].str.replace('\n', '')
df2["label"] = 1
print(df2.head())

                                                text  label
0  When considering the pros and cons of attendin...      1
1  One of the primary arguments for implementing ...      1
2  Before making any decisions about getting invo...      1
3  Ralph Waldo Emerson once said, "Go confidently...      1
4  Human character traits are shaped by a wide ra...      1


In [ ]:
df=pd.concat([df1,df2],axis=0)
df['label'].value_counts()

0    14250
1     4042
Name: label, dtype: int64

In [ ]:
df.shape

(18292, 2)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier

In [ ]:
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 3),sublinear_tf=True)
X= tfidf_vectorizer.fit_transform(df['text'])

In [ ]:
y=np.array(df['label'])
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=500)

In [ ]:
lr_model = LogisticRegression()
sgd_model = SGDClassifier(max_iter=1000, tol=1e-3, loss="modified_huber")

In [ ]:
from sklearn.metrics import accuracy_score
lr_model.fit(X_train,y_train)
y_pred =lr_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9850919330793441

In [ ]:
from sklearn.metrics import accuracy_score
sgd_model.fit(X_train,y_train)
y_pred =sgd_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.9968527414278615

lr_model = LogisticRegression()
sgd_model = SGDClassifier(max_iter=1000, tol=1e-3, loss="modified_huber")

In [ ]:
ensemble_model= VotingClassifier(estimators=[('lr', lr_model),('sgd', sgd_model)], voting='soft')
ensemble_model.fit(X_train,y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('sgd', SGDClassifier(loss='modified_huber'))],
                 voting='soft')

In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# dt_model = DecisionTreeClassifier(random_state=42)


# dt_model.fit(X_train, y_train)

# y_pred = dt_model.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# accuracy

In [ ]:
# from sklearn.ensemble import RandomForestClassifier

# rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_model.fit(X_train, y_train)

# y_pred = rf_model.predict(X_test)

# accuracy = accuracy_score(y_test, y_pred)
# accuracy

In [ ]:
preds_test = ensemble_model.predict_proba(X_test)[:,1]
print(preds_test)

[0.01211999 0.0207894  0.9637043  ... 0.02741314 0.94372561 0.02374827]


In [ ]:
preds_test_binary = np.where(preds_test > 0.5, 1, 0)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,preds_test_binary)

0.9917177405996356

In [ ]:

test = [
    '''terrorism has led to loss of innocent civilians but more importantly those soldiers who are serving for the betterment of our country . if they wont be there our country would not be safe . respect those who are awake so that you may sleep properly. Many young blood are nowadays willing to recruit in the army and it is one of the best thing that not all of chasing money but there are also some people who are willing to serve the country''',
''' In a world where choices abound, from the cereal aisle in the grocery store to career paths and life partners, the concept of the paradox of choice becomes increasingly relevant. This essay delves into the complexities of decision-making in the modern era, exploring how an abundance of options can lead to both liberation and, paradoxically, increased stress and dissatisfaction.
The paradox of choice, coined by psychologist Barry Schwartz, suggests that while having more choices theoretically provides freedom and autonomy, it can also overwhelm individuals, leading to decision fatigue and a sense of dissatisfaction. The essay will dissect this paradox by examining its manifestations in various aspects of life, such as consumerism, career decisions, and relationships.'''
,"In every place of great resort the monster was the fashion. They sang of it in the cafes, ridiculed it in the papers, and represented it on the stage. All kinds of stories were circulated regarding it. There appeared in the papers caricatures of every gigantic and imaginary creature, from the white whale, the terrible 'Moby Dick' of sub-arctic regions, to the immense kraken, whose tentacles could entangle a ship of five hundred tons and hurry it into the abyss of the ocean. The legends of ancient times were even revived.Then burst forth the unending argument between the believers and the unbelievers in the societies of the wise and the scientific journals. 'The question of the monster' inflamed all minds. Editors of scientific journals, quarrelling with believers in the supernatural, spilled seas of ink during this memorable campaign, some even drawing blood; for from the sea-serpent they came to direct personalities.",
'''But the names are but locationally appropriate. Hell is no fuming,
torrid city. It is temperate with a perfect climate. Mephisto's only
claim to the nether regions was the dancing flames of her smelting
mills that danced on the night sky. The Devil's Range was a small ridge
of less than fifteen thousand feet and it was more than amply supplied
with passes and near-sea-level breaches.

And the cities at the mouth of the River Styx lived in cheerful
rivalry, their main source of jealousy being the lush produce that
came from the hinterland behind each. And the River Styx itself was a
garden-spot for yachting clubs; bathing beaches lined the mouth for
fifteen miles inward and they were clear-watered and pearly sanded.

Pluto had been a man-made paradise for a number of years, only because
Man, the Adaptable, found it economically expedient to make it so.'''
 , "I was perfectly up in the subject which was the question of the day. How could I be otherwise? I had read and reread all the American and European papers without being any nearer a conclusion. This mystery puzzled me. Under the impossibility of forming an opinion, I jumped from one extreme to the other. That there really was something could not be doubted, and the incredulous were invited to put their finger on the wound of the Scotia.On my arrival at New York the question was at its height. The theory of the floating island, and the unapproachable sandbank, supported by minds little competent to form a judgment, was abandoned. And, indeed, unless this shoal had a machine in its stomach, how could it change its position with such astonishing rapidity?"
 ,'''As technology continues to advance, the integration of artificial intelligence (AI) in healthcare has become both a source of great promise and a subject of ethical scrutiny. This essay explores the profound impact of AI on the healthcare industry, examining the ethical challenges it presents, as well as the potential benefits it offers to patients, medical professionals, and society at large.

AI in healthcare encompasses a range of applications, from diagnostic tools and treatment recommendations to personalized medicine and administrative tasks. The essay will delve into the ethical considerations surrounding patient privacy, data security, and the potential biases embedded in AI algorithms.'''
,'''To check if a text is AI-generated, we need to compare the sentence perplexity with the model’s perplexity scaled by a fudge-factor, alpha. If the sentence perplexity is more than the model’s perplexity with scaling, then it’s probably human-written text (i.e. not AI-generated). Otherwise, it’s probably AI-generated. The reason for this is that we expect the model to not be perplexed by text it would generate itself, so if it encounters some text that it itself would not generate, then there’s reason to believe that the text isn’t AI-generated. If the perplexity of the sentence is less than or equal to the model’s training perplexity with scaling, then it’s likely that it was generated using this language model, but we can’t be very sure. This is because it’s possible for a human to have written that text, and it just happens to be something that the model could also have generated. After all, the model was trained on a lot of human-written text so in some sense, the model represents an “average human’s writing”.'''
,'''In the realm of human consciousness, the boundaries between dreams and reality often blur, creating a fascinating interplay that shapes our perceptions and experiences. This intersection of the conscious and the subconscious has intrigued thinkers, artists, and philosophers throughout the ages. The concept of the interconnectedness of dreams and reality invites us to explore the profound ways in which our inner worlds influence the external narrative of our lives.

Dreams, those ethereal landscapes woven by the subconscious mind during periods of sleep, have long captivated the human imagination. In these alternate realities, the laws of physics bend, and the inconceivable becomes conceivable. Dreams serve as a playground for our deepest desires, fears, and aspirations. They act as a canvas where the mind is free to paint vivid landscapes, tell untold stories, and confront unresolved emotions. As we navigate the landscapes of our dreams, we unearth the enigmatic aspects of our psyche, revealing hidden facets of our identity.''']



In [ ]:
test_x=tfidf_vectorizer.transform(test)

In [ ]:
results=ensemble_model.predict_proba(test_x)[:,1]
for i in results:
  if (i>0.5 and i<0.6):
       print(i)
       print("MAYBE M IX OF AI AND HUMAN")
  elif i>0.6:
       print(i)
       print("AI GENERATED")
  else:
       print(i)
       print("HUMAN GENERATED")

0.2590029889326564
HUMAN GENERATED
0.9011701111327162
AI GENERATED
0.38909962907548795
HUMAN GENERATED
0.2908760505403253
HUMAN GENERATED
0.27103573716568463
HUMAN GENERATED
0.9050937716529877
AI GENERATED
0.05008894215378171
HUMAN GENERATED
0.6557024266157537
AI GENERATED


In [ ]:
!pip install joblib

In [ ]:
import joblib

In [ ]:

joblib.dump(ensemble_model, 'ensemble_model.joblib')

['ensemble_model.joblib']

In [ ]:
joblib.dump(tfidf_vectorizer, 'tfidf_vectorizer.joblib')

['tfidf_vectorizer.joblib']